In [1]:
cd '/home/gus/Documentos/mestrado/dados-mestrado/vento-inmet'

/home/gus/Documentos/mestrado/dados-mestrado/vento-inmet


In [2]:
!ls

dados_header.csv		    vento_bruto.csv
frequencia_direcao.csv		    vento_limpo.csv
frequencia_direcao_e_magnitude.csv  vento_medias_mensais.csv
frequencia_magnitude.csv	    vento_rotacionado.csv
frequencia_rajada.csv		    vento_std_mensal.csv
valores_ventos_mensais_.ods


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
#import matplotlib.dates as mdates
#from windrose import WindroseAxes

<h2> Quadrante </h2>
<p>Graus (º)</p>
<p>Norte 337º a 360º e 0º a 22º</p>
<p>NE 23º a 67º</p>
<p>Leste 68º a 112º</p>
<p>SE 113º a 157º</p>
<p>Sul 158º a 202º</p>
<p>SW 203º a 247º</p>
<p>Oeste 248º a 292º</p>
<p>NW 293º a 337º</p>

In [4]:
df = pd.read_csv('vento_limpo.csv', delimiter=';',
                 na_values = -99999.0)

In [5]:
df.dropna(inplace=True)

In [6]:
df.head()

,direction,wg,w,time
0,92.0,4.369711,1.627932,2016-12-01 00:00:00
1,86.0,4.969476,2.142015,2016-12-01 01:00:00
2,92.0,5.312198,2.056335,2016-12-01 02:00:00
3,87.0,5.997643,2.056335,2016-12-01 03:00:00
4,93.0,6.426046,2.998821,2016-12-01 04:00:00


In [7]:
# convertendo string para datetime
time = [dt.datetime.strptime(ii, "%Y-%m-%d %H:%M:%S") for ii in df.time]

In [8]:
df.time = time
del time

In [9]:
df['year'] = df.time.dt.year
df['month'] = df.time.dt.month

### agrupando os dados em classes

##### direção

In [10]:
octante = []
for linha in range(8699):

    if 0<= df.iloc[linha, 0] < 23:
        octante.append("N")

    elif 23 <= df.iloc[linha, 0] < 67:
        octante.append("NE")

    elif 67 <= df.iloc[linha, 0] < 112:
        octante.append("E")

    elif 112 <= df.iloc[linha, 0] < 157:
        octante.append("SE")

    elif 157<= df.iloc[linha, 0] < 202:
        octante.append("S")

    elif 202<= df.iloc[linha, 0] < 247:
        octante.append("SW")

    elif 247 <= df.iloc[linha, 0] < 292:
        octante.append("W")

    elif 292 <= df.iloc[linha, 0] < 337:
        octante.append("NW")
    elif 337<=df.iloc[linha,0]<= 360:
        octante.append("N")


##### rajada

In [11]:
gust_class = []

for linha in range(8699):

    if 0 <= df.iloc[linha, 1] <= 2:
        gust_class.append("0-2")

    elif 2 < df.iloc[linha, 1] <= 4:
        gust_class.append("2-4")

    elif 4 < df.iloc[linha, 1] <= 6:
        gust_class.append("4-6")

    elif 6 < df.iloc[linha, 1] <= 8:
        gust_class.append("6-8")
    
    elif 8 < df.iloc[linha, 1] <= 10:
        gust_class.append("8-10")
        
    elif 10 < df.iloc[linha, 1] <= 12:
        gust_class.append("10-12")
        
    elif 12 < df.iloc[linha, 1] <= 14:
        gust_class.append("12-14")
        
    elif 14 < df.iloc[linha, 1]:
        gust_class.append("+14")  
        


In [12]:
interval = []

for linha in range(8699):

    if 0 <= df.iloc[linha, 2] <= 2:
        interval.append("0-2")

    elif 2 < df.iloc[linha, 2] <= 4:
        interval.append("2-4")

    elif 4 < df.iloc[linha, 2] <= 6:
        interval.append("4-6")

    elif 6 < df.iloc[linha, 2] <= 8:
        interval.append("6-8")
    else:
        interval.append("8+")

In [13]:
rc = pd.DataFrame({"octante":octante, "intervalo_vento": interval,
                   'intervalo_rajada': gust_class})

In [14]:
rc['year'] = df.time.dt.year.copy()
rc['month'] = df.time.dt.month.copy()

### frequncia total

In [15]:
rc.intervalo_rajada.value_counts()/rc.intervalo_rajada.value_counts().sum()

6-8      0.286010
4-6      0.275204
2-4      0.166571
8-10     0.150017
0-2      0.098057
10-12    0.022991
12-14    0.001150
Name: intervalo_rajada, dtype: float64

In [16]:
rc.intervalo_vento.value_counts()/rc.intervalo_vento.value_counts().sum()

2-4    0.476836
0-2    0.474537
4-6    0.048626
Name: intervalo_vento, dtype: float64

In [17]:
rc.octante.value_counts()/rc.octante.value_counts().sum()

E     0.494080
SE    0.257386
S     0.198644
SW    0.020117
NE    0.018738
NW    0.004598
W     0.004023
N     0.002414
Name: octante, dtype: float64

### frequencia chuvoso

In [18]:
rc_chuvoso = rc.loc[(rc.month>=3)&\
                     (rc.month<=5),:].copy()

In [19]:
rc_chuvoso.octante.value_counts()/rc_chuvoso.octante.value_counts().sum()

E     0.354136
S     0.293710
SE    0.232789
SW    0.054978
NE    0.033185
NW    0.013868
W     0.010896
N     0.006439
Name: octante, dtype: float64

In [20]:
rc_chuvoso.intervalo_vento.value_counts()/rc_chuvoso.intervalo_vento.value_counts().sum()

0-2    0.725607
2-4    0.269936
4-6    0.004458
Name: intervalo_vento, dtype: float64

In [21]:
rc_chuvoso.intervalo_rajada.value_counts()/rc_chuvoso.intervalo_rajada.value_counts().sum()

2-4     0.283804
0-2     0.251114
4-6     0.239227
6-8     0.191184
8-10    0.034671
Name: intervalo_rajada, dtype: float64

### frequencia seco

In [22]:
rc_seco = rc.loc[(rc.month>=8)&\
                     (rc.month<=10),:].copy()

In [23]:
rc_seco.octante.value_counts()/rc_seco.octante.value_counts().sum()

E     0.549901
SE    0.367095
S     0.082510
NE    0.000494
Name: octante, dtype: float64

In [24]:
rc_seco.intervalo_vento.value_counts()/rc_seco.intervalo_vento.value_counts().sum()

2-4    0.667984
0-2    0.195652
4-6    0.136364
Name: intervalo_vento, dtype: float64

In [25]:
rc_seco.intervalo_rajada.value_counts()/rc_seco.intervalo_rajada.value_counts().sum()

6-8      0.348320
8-10     0.320652
4-6      0.199111
10-12    0.070158
2-4      0.050889
0-2      0.006423
12-14    0.004447
Name: intervalo_rajada, dtype: float64